Detect face from low quality video footage and increase the image quality.




In [1]:
!pip install deepface
!pip install tqdm
!pip install tensorflow-hub

In [2]:
from google.colab import files
import cv2
from deepface import DeepFace
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
esrgan_model = hub.load('https://tfhub.dev/captain-pool/esrgan-tf2/1')

In [4]:
import tensorflow as tf

# Check if a GPU is available and print the device name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
# Ask the user to upload a video file
uploaded = files.upload()
video_file_name = next(iter(uploaded))

Saving berzin ka.mp4 to berzin ka (1).mp4


In [ ]:
# Function to preprocess the frames for ESRGAN
def preprocess_frame(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_rgb = tf.cast(frame_rgb, tf.float32) / 255.0
    frame_rgb = tf.expand_dims(frame_rgb, 0)
    return frame_rgb

# Function to upscale images using a pre-trained ESRGAN model
def upscale_frame(frame, model):
    frame_preprocessed = preprocess_frame(frame)
    upscaled_frame = model(frame_preprocessed)
    upscaled_frame = tf.squeeze(upscaled_frame)
    upscaled_frame = tf.clip_by_value(upscaled_frame, 0, 1)
    upscaled_frame = tf.cast(upscaled_frame * 255, tf.uint8)
    upscaled_frame = cv2.cvtColor(upscaled_frame.numpy(), cv2.COLOR_RGB2BGR)
    return upscaled_frame

# Function to extract and display faces from a video
def extract_and_display_faces(video_path):
    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    face_count = 0
    detected_faces = []
    pbar = tqdm(total=total_frames, desc="Processing frames")

    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break

        upscaled_frame = upscale_frame(frame, esrgan_model)
        rgb_frame = cv2.cvtColor(upscaled_frame, cv2.COLOR_BGR2RGB)

        try:
            # Use extract_faces instead of the deprecated detectFace
            faces = DeepFace.extract_faces(rgb_frame, detector_backend='opencv', enforce_detection=False)
            detected_faces.extend(faces)
            face_count += len(faces)
        except Exception as e:
            print(f"Error: {e}")

        pbar.update(1)

    video.release()
    pbar.close()
    print(f"Total frames: {total_frames}, Faces found: {face_count}")

    for face in detected_faces:
        plt.imshow(face)
        plt.axis('off')
        plt.show()

# Call the function with the uploaded video file
extract_and_display_faces(video_file_name)

Processing frames:   0%|          | 0/3840 [00:00<?, ?it/s]